In [0]:
from keras.models import model_from_json
from pathlib import Path
from keras.preprocessing import image
import numpy as np
from keras.applications import vgg16

In [0]:
# Load the json file that contains the model's structure
f = Path("model_structure.json")
model_structure = f.read_text()

# Recreate the Keras model object from the json data
model = model_from_json(model_structure)

# Re-load the model's trained weights
model.load_weights("model_weights.h5")

In [34]:
model.summary()

Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_32 (Flatten)         (None, 2048)              0         
_________________________________________________________________
dense_88 (Dense)             (None, 256)               524544    
_________________________________________________________________
dropout_50 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_89 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_51 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_90 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_52 (Dropout)         (None, 256)             

In [35]:
import pandas as pd
cnt = 0
correct = 0
images_path = Path("test_r_set")
for img,detail in zip(images_path.glob("*.jpg"),images_path.glob("*.json")):
  # Load an image file to test, resizing it to 64x64 pixels (as required by this model)
  img = image.load_img(img, target_size=(64, 64))
  label = pd.read_json(detail)
  # Convert the image to a numpy array
  image_array = np.array(img)
  #image_array = image.img_to_array(img)

  # Add a forth dimension to the image (since Keras expects a bunch of images, not a single image)
  images = np.expand_dims(image_array, axis=0)

  # Normalize the data
  images = vgg16.preprocess_input(images)
  # Use the pre-trained neural network to extract features from our test image (the same way we did to train the model)
  feature_extraction_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
  features = feature_extraction_model.predict(images)
  # Given the extracted features, make a final prediction using our own model
  results = model.predict(features)

  # Since we are only testing one image with possible class, we only need to check the first result's first element
  single_result = results[0][0]

  # Print the result
  print("Likelihood that this skin cancer is malignant: {}%".format(int(single_result * 100)))
  print("Likelihood that this skin cancer is benign: {}%".format(int(100 - single_result * 100)))
  print(int(label.loc['clinical','meta']['benign_malignant'] == 'malignant'))
  
  if((int(single_result * 100)>=50 and label.loc['clinical','meta']['benign_malignant'] == 'malignant') or (int(single_result * 100)<50 and int(label.loc['clinical','meta']['benign_malignant'] != 'malignant')==1) ):
    correct+=1
  cnt+=1

Likelihood that this skin cancer is malignant: 38%
Likelihood that this skin cancer is benign: 61%
0
Likelihood that this skin cancer is malignant: 2%
Likelihood that this skin cancer is benign: 97%
1
Likelihood that this skin cancer is malignant: 40%
Likelihood that this skin cancer is benign: 59%
0
Likelihood that this skin cancer is malignant: 1%
Likelihood that this skin cancer is benign: 98%
1
Likelihood that this skin cancer is malignant: 0%
Likelihood that this skin cancer is benign: 99%
1
Likelihood that this skin cancer is malignant: 4%
Likelihood that this skin cancer is benign: 95%
0
Likelihood that this skin cancer is malignant: 4%
Likelihood that this skin cancer is benign: 95%
0
Likelihood that this skin cancer is malignant: 0%
Likelihood that this skin cancer is benign: 99%
0
Likelihood that this skin cancer is malignant: 12%
Likelihood that this skin cancer is benign: 87%
0
Likelihood that this skin cancer is malignant: 9%
Likelihood that this skin cancer is benign: 90%

ResourceExhaustedError: ignored

In [36]:
correct/cnt

0.7027027027027027